In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
import json

## scrape

### 1.1 mapping

In [24]:
links = []
for page in range(1,201,1):
    url = f'https://www.boligsiden.dk/tilsalg?sortAscending=true&page={page}'
    links.append(url)

In [25]:
links

['https://www.boligsiden.dk/tilsalg?sortAscending=true&page=1',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=2',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=3',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=4',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=5',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=6',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=7',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=8',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=9',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=10',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=11',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=12',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=13',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=14',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=15',
 'https://www.boligsiden.dk/tilsal

In [26]:
# Create a new list
houses = []
for url in links:
    # Connects to site
    response = requests.get(url, headers={'name':'He SHI','email':'pvs237@alumni.ku.dk'})
    
    #Parse data with BeautifulSoup 
    soup = BeautifulSoup(response.content, 'lxml')

    # Identify houses to scrape by inspecting site 
    houses += soup.find_all('div', class_ = 'overflow-hidden relative shadow-card rounded-sm grid grid-cols-1 grid-rows-1 sm:grid-cols-12 sm:grid-rows-1 border border-gray-100 bg-background')

    # sleep for 0.5 seconds
    time.sleep(0.5) #Sleep for 0.5 seconds

In [27]:
houses
len(houses)

10000

We then create a list of URLs that we want to scrape

In [28]:
# Create an empty list
list_of_house_urls = []

# Creating a loop that appends the article url to the list above
for i in range(len(houses)):
    list_of_house_urls.append(houses[i].find('a')['href'])

In [29]:
list_of_house_urls

['/adresse/horsekildevej-26-0-th-2500-valby-01013028__26_st__th?udbud=4e6f9d84-e3f4-4b15-b473-b671e09a28a1',
 '/adresse/sandtoftevej-36-6857-blaavand-05731074__36_______?udbud=7b620270-26d8-44dd-8357-a017e29e96d5',
 '/viderestillingaabenthus/39f3036c-95a6-401f-827e-d940121b746e',
 '/adresse/frederikssundsvej-33a-3300-frederiksvaerk-02600360_33a_______?udbud=5a6b96c3-d90d-4dd8-af11-7cd3b6f217eb',
 '/adresse/holmegaardsvej-21-4684-holmegaard-03700666__21_______?udbud=0e4e56c0-c909-46be-9221-6fa616f9d89d',
 '/adresse/tollestrupvej-55a-9500-hobro-08461643_55a_______?udbud=7db01c8b-4174-46ff-8850-3322ea7df66c',
 '/adresse/nattergalevej-42-3-tv-2400-koebenhavn-nv-01014948__42__3__tv?udbud=a138df6c-9f50-4a8d-9683-a038f36000ef',
 '/adresse/landevejen-59-4684-holmegaard-03700938__59_______?udbud=ce756cc5-d0a9-481e-a772-a94ce4df45e0',
 '/adresse/roedegaardsvej-47-2-tv-5000-odense-c-04616902__47__2__tv?udbud=6e247c2a-9419-4dbc-b52b-d23a4e7200df',
 '/viderestillingaabenthus/40134eb9-edae-44c0-9b0f

Some of the links are not to articls, then we...

In [30]:
list_of_house_urls_final = []
for link in list_of_house_urls:
    if '/adresse' in link: #All article URLs have this string in them, so we restrict on it being in the URL
        list_of_house_urls_final.append(link)

### 1.2 Downloading + 1.3 Parsing

In [77]:
# Define the log function to gather the log information
def log(response,logfile,output_path=os.getcwd()):
    # Open or create the csv file
    if os.path.isfile(logfile): #If the log file exists, open it and allow for changes     
        log = open(logfile,'a')
    else: #If the log file does not exist, create it and make headers for the log variables
        log = open(logfile,'w')
        header = ['timestamp','status_code','length','output_file']
        log.write(';'.join(header) + "\n") #Make the headers and jump to new line
        
    # Gather log information
    status_code = response.status_code #Status code from the request result
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())) #Local time
    length = len(response.text) #Length of the HTML-string
    
    # Open the log file and append the gathered log information
    with open(logfile,'a') as log:
        log.write(f'{timestamp};{status_code};{length};{output_path}' + "\n") #Append the information and jump to new line

[None]

In [ ]:
# Creatig empty list for the infomation we want to extract for every article
data_list = []
address_list = []
area_list = []
city_list = []
type_list = []
price_list = []
logfile = 'log.csv'
for i in range(7675): #len(list_of_house_urls_final)
    try:
        # This time we scrape for each house in the url list we created before
        url = 'https://www.boligsiden.dk' + list_of_house_urls_final[i] #The scraped links are relative, so we need to add the base url
        response = requests.get(url)
        soup = BeautifulSoup(response.content,'lxml')
    
        # Append price to list
        temp = soup.find('div', class_='flex flex-row justify-center items-center space-x-2')
        temp = temp.text.strip()
        price_list.append(temp)

        # Append address to list
        temp = soup.find('div', class_='font-bold text-sm md:text-base') 
        temp = temp.text.strip()
        address_list.append(temp)
    
        # Append area to list
        temp = soup.find('div', class_='mt-1 text-xs md:text-sm text-gray-600')
        temp = temp.text.strip()
        area_list.append(temp)

        # Append city to list
        temp = soup.find('span', class_='inline-flex')
        temp = temp.text.strip()
        city_list.append(temp)

        # Append type to list
        temp = soup.find('span', class_='text-black text-sm pr-2')
        temp = temp.text.strip()
        type_list.append(temp)

        # Append other data to list 
        temp = soup.find('div', class_='py-5 px-2 md:px-6 grid text-sm grid-cols-2')
        temp = temp.text.strip()
        data_list.append(temp)

        # Time sleep
        time.sleep(0.5) #Sleep for 0.5 seconds

        # Logging
        log(response,logfile)

    except Exception as e:
        print(url) #Print url
        print(e) #Print error
        continue #Continue to next iteration of the loop
